In [51]:
pwd

'/Users/user/Desktop/2023python'

In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import (DataLoader,)
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

In [40]:
class CNN(nn.Module):
    def __init__(self,in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        #1 layer
        self.conv1 = nn.Conv2d( 
            in_channels = in_channels,
            out_channels = 8,
            kernel_size =3,
            stride =1,
            padding=1        
        )
        #2 max pool
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        #3 2nd conv
        self.conv2 =nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        
        self.fc1=nn.Linear(16*7*7, num_classes)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        return x
        

In [41]:
CNN()

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [43]:
#hyperparameters
in_channels =1
num_classes =10
learning_rate =3e-4 #karapathy's constant
batch_size =64
num_epochs =3

In [44]:
train_dataset = datasets.MNIST( 
root ="dataset/" , train=True, transform=transforms.ToTensor(),download =True
)

In [45]:
test_dataset = datasets.MNIST(
root="dataset/", train=False, transform=transforms.ToTensor(), download =True
)

In [46]:
train_loader =DataLoader(dataset =train_dataset,batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [47]:
model=CNN(in_channels=in_channels, num_classes=num_classes).to(device)

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer =optim.Adam(model.parameters(), lr=learning_rate)

In [49]:
for epoc in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        
        #get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        #forward
        scores =model(data) #output
        loss = criterion(scores, targets) #loss
        
        #backward
        optimizer.zero_grad() #init to zeros
        loss.backward() #calculate loss backwards
        
        #gradient descent or adam step
        optimizer.step()

100%|██████████| 938/938 [00:25<00:00, 36.61it/s]


In [50]:
def check_accuracy(loader,model):
    num_correct =0
    num_samples =0 
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x=x.to(device=device)
            y=y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 95.58
Accuracy on test set: 95.74
